# ПРАКТИЧЕСКАЯ РАБОТА №3
### _Сверточные нейронные сети для анализа данных_

## Теоретическая часть

### Введение в сверточные нейронные сети (CNN)

**Сверточные нейронные сети** - это специальный тип нейронных сетей, предназначенный для обработки данных с сеточной структурой (изображения, временные ряды, спектрограммы).

Основные компоненты CNN:
- **Сверточные слои** - извлечение локальных特征
- **Пулинговые слои** - уменьшение размерности
- **Полносвязные слои** - классификация
- **Функции активации** - ReLU, Softmax

### Области применения:
- Классификация изображений
- Обработка временных рядов
- Анализ спектральных данных
- Распознавание образов

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Библиотеки для нейронных сетей
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# Настройка отображения
plt.style.use('ggplot')
sns.set_palette("husl")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## Практическая часть: Создание CNN для анализа данных ирисов

### Задача 1: Подготовка данных для CNN

In [ ]:
# Загрузка и анализ данных
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print("Исходные данные:")
print(f"Размерность признаков: {X.shape}")
print(f"Количество классов: {len(target_names)}")
print(f"Названия классов: {target_names}")
print(f"Признаки: {feature_names}")

# Создаем DataFrame для анализа
df_iris = pd.DataFrame(X, columns=feature_names)
df_iris['target'] = y
df_iris['species'] = [target_names[i] for i in y]

print("\nПервые 5 строк данных:")
df_iris.head()

In [ ]:
# Визуализация распределения данных
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Распределение классов
df_iris['species'].value_counts().plot(kind='bar', ax=axes[0,0], color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0,0].set_title('Распределение видов ирисов')
axes[0,0].set_ylabel('Количество')

# Корреляционная матрица
sns.heatmap(df_iris[feature_names].corr(), annot=True, cmap='coolwarm', ax=axes[0,1])
axes[0,1].set_title('Корреляционная матрица признаков')

# Boxplot признаков
df_iris[feature_names].boxplot(ax=axes[1,0])
axes[1,0].set_title('Распределение признаков')
axes[1,0].tick_params(axis='x', rotation=45)

# Scatter plot
scatter = axes[1,1].scatter(df_iris['petal length (cm)'], df_iris['petal width (cm)'], 
                           c=df_iris['target'], cmap='viridis')
axes[1,1].set_xlabel('Petal Length (cm)')
axes[1,1].set_ylabel('Petal Width (cm)')
axes[1,1].set_title('Petal Length vs Petal Width')
plt.colorbar(scatter, ax=axes[1,1])

plt.tight_layout()
plt.show()

### Задача 2: Преобразование данных для CNN

In [ ]:
# Преобразование данных в формат, подходящий для CNN
# Создаем "псевдо-изображения" из данных

def create_sequence_data(X, sequence_length=4):
    """Преобразование данных в последовательности для CNN"""
    X_sequence = X.reshape(X.shape[0], sequence_length, 1)
    return X_sequence

# Преобразуем данные
X_sequence = create_sequence_data(X)

print("До преобразования:")
print(f"Форма X: {X.shape}")
print("\nПосле преобразования:")
print(f"Форма X_sequence: {X_sequence.shape}")
print(f"Пример последовательности для первого образца: {X_sequence[0].flatten()}")

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_sequence, y, test_size=0.3, random_state=42, stratify=y
)

# Масштабирование данных
scaler = StandardScaler()
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat = scaler.transform(X_test_flat)

X_train_scaled = X_train_scaled_flat.reshape(X_train.shape)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape)

# Преобразование меток в one-hot encoding
y_train_categorical = to_categorical(y_train, num_classes=3)
y_test_categorical = to_categorical(y_test, num_classes=3)

print("Размерности данных:")
print(f"X_train: {X_train_scaled.shape}")
print(f"X_test: {X_test_scaled.shape}")
print(f"y_train: {y_train_categorical.shape}")
print(f"y_test: {y_test_categorical.shape}")

### Задача 3: Создание и обучение сверточной нейронной сети

In [ ]:
# Создание модели CNN
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        # Первый сверточный слой
        layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        
        # Второй сверточный слой
        layers.Conv1D(64, kernel_size=2, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        
        # Выравнивание и полносвязные слои
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Создаем модель
input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
num_classes = 3

cnn_model = create_cnn_model(input_shape, num_classes)

# Компиляция модели
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Архитектура CNN модели:")
cnn_model.summary()

In [ ]:
# Обучение модели
history = cnn_model.fit(
    X_train_scaled, y_train_categorical,
    epochs=100,
    batch_size=8,
    validation_data=(X_test_scaled, y_test_categorical),
    verbose=1
)

### Задача 4: Оценка качества модели

In [ ]:
# Визуализация процесса обучения
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# График точности
axes[0].plot(history.history['accuracy'], label='Обучающая точность')
axes[0].plot(history.history['val_accuracy'], label='Валидационная точность')
axes[0].set_title('Точность модели')
axes[0].set_xlabel('Эпоха')
axes[0].set_ylabel('Точность')
axes[0].legend()
axes[0].grid(True)

# График потерь
axes[1].plot(history.history['loss'], label='Обучающие потери')
axes[1].plot(history.history['val_loss'], label='Валидационные потери')
axes[1].set_title('Потери модели')
axes[1].set_xlabel('Эпоха')
axes[1].set_ylabel('Потери')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Предсказания и оценка модели
y_pred_proba = cnn_model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_proba, axis=1)

# Метрики качества
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность CNN модели: {accuracy:.3f}")

print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=target_names,
            yticklabels=target_names)
plt.title('Матрица ошибок CNN модели')
plt.ylabel('Истинные значения')
plt.xlabel('Предсказанные значения')
plt.show()

### Задача 5: Сравнение с полносвязной нейронной сетью

In [ ]:
# Создание полносвязной нейронной сети для сравнения
def create_dnn_model(input_dim, num_classes):
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(16, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Подготовка данных для DNN
X_train_dnn = X_train_scaled.reshape(X_train_scaled.shape[0], -1)
X_test_dnn = X_test_scaled.reshape(X_test_scaled.shape[0], -1)

# Создание и обучение DNN
dnn_model = create_dnn_model(X_train_dnn.shape[1], num_classes)
dnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_dnn = dnn_model.fit(
    X_train_dnn, y_train_categorical,
    epochs=100,
    batch_size=8,
    validation_data=(X_test_dnn, y_test_categorical),
    verbose=0
)

# Предсказания DNN
y_pred_dnn = np.argmax(dnn_model.predict(X_test_dnn), axis=1)
accuracy_dnn = accuracy_score(y_test, y_pred_dnn)

print("СРАВНЕНИЕ МОДЕЛЕЙ:")
print(f"Точность CNN: {accuracy:.3f}")
print(f"Точность DNN: {accuracy_dnn:.3f}")

# Сравнение архитектур
print("\nСРАВНЕНИЕ АРХИТЕКТУР:")
print("CNN - количество параметров:", cnn_model.count_params())
print("DNN - количество параметров:", dnn_model.count_params())

# ЗАДАНИЕ ДЛЯ ВЫПОЛНЕНИЯ

### Тема: "Сверточные нейронные сети для анализа различных типов данных"

**Цель:** Исследовать применение CNN для анализа разных типов данных и сравнить эффективность различных архитектур.

### Задание 1: Анализ временных рядов с помощью CNN

**Задача:** Использовать CNN для классификации временных рядов на примере датасета ECG

**Подзадачи:**
1. Загрузите датасет временных рядов (согласно варианту)
2. Подготовьте данные для CNN (нормализация, создание последовательностей)
3. Создайте CNN архитектуру для анализа временных рядов
4. Обучите модель и оцените ее качество
5. Сравните с другими методами классификации временных рядов

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. Загрузка данных
CSV_FILE = "train.csv"
data = pd.read_csv(CSV_FILE)

# Обработка пропусков
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Кодировка
data['Sex'] = LabelEncoder().fit_transform(data['Sex'])
data['Embarked'] = LabelEncoder().fit_transform(data['Embarked'])

# Признаки
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = data[features].values
y = data['Survived'].values

# Нормализация
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# reshape для Conv1D
X_seq = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# train/test
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.2, random_state=62)

# CNN модель
model = Sequential([
    Conv1D(32, 2, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

# Сравнение с классическими методами
X_train_flat = X_train.reshape(X_train.shape[0], X_train.shape[1])
X_test_flat = X_test.reshape(X_test.shape[0], X_test.shape[1])

lr = LogisticRegression(max_iter=200)
lr.fit(X_train_flat, y_train)
print("LogReg Accuracy:", accuracy_score(y_test, lr.predict(X_test_flat)))

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_flat, y_train)
print("Random Forest Accuracy:", accuracy_score(y_test, rf.predict(X_test_flat)))


### Задание 2: CNN для многомерных данных

**Задача:** Разработать CNN для анализа многомерных данных из выбранного датасета

**Подзадачи:**
1. Выберите многомерный датасет (согласно варианту)
2. Преобразуйте данные в формат, подходящий для CNN (2D или 3D структуры)
3. Создайте и обучите несколько архитектур CNN
4. Проведите оптимизацию гиперпараметров
5. Проанализируйте влияние глубины сети на качество классификации

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 1. ЗАГРУЗКА ДАТАСЕТА

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

print("Размеры:", X_train.shape, y_train.shape)   # (60000, 28, 28)

# 2. ПРЕОБРАЗОВАНИЕ ДАННЫХ ДЛЯ CNN

# Добавление канала (28, 28) -> (28, 28, 1)
X_train = X_train.astype("float32") / 255.0
X_test  = X_test.astype("float32")  / 255.0

X_train = np.expand_dims(X_train, -1)
X_test  = np.expand_dims(X_test, -1)

# Разделяем на train/val
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)

# One-hot encoding
y_train_oh = to_categorical(y_train, 10)
y_val_oh   = to_categorical(y_val,   10)
y_test_oh  = to_categorical(y_test,  10)

# 3. СОЗДАНИЕ НЕСКОЛЬКИХ АРХИТЕКТУР CNN

def build_cnn_model(depth=2, filters=32):
    """
    depth — количество блоков Conv+ReLU+Pool
    filters — базовое число фильтров
    """
    model = models.Sequential()
    model.add(layers.Input(shape=(28, 28, 1)))

    for i in range(depth):
        model.add(layers.Conv2D(filters * (2**i), (3,3), activation='relu', padding="same"))
        model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


# Пример трёх моделей разной сложности
models_to_train = {
    "CNN_small":  build_cnn_model(depth=1, filters=16),
    "CNN_medium": build_cnn_model(depth=2, filters=32),
    "CNN_large":  build_cnn_model(depth=3, filters=32),
}


history_dict = {}

for name, model in models_to_train.items():
    print(f"\n==== Обучение модели {name} ====")
    history = model.fit(
        X_train, y_train_oh,
        validation_data=(X_val, y_val_oh),
        epochs=5, batch_size=64, verbose=1
    )
    history_dict[name] = history

# 4. ПРОСТАЯ ОПТИМИЗАЦИЯ ГИПЕРПАРАМЕТРОВ
def grid_search():
    depths = [1, 2, 3]
    filters = [16, 32, 64]

    best_acc = 0
    best_params = None

    for d in depths:
        for f in filters:
            print(f"\nПробую depth={d}, filters={f}")
            model = build_cnn_model(depth=d, filters=f)
            history = model.fit(
                X_train, y_train_oh,
                validation_data=(X_val, y_val_oh),
                epochs=3, batch_size=64, verbose=0
            )
            acc = history.history["val_accuracy"][-1]

            print(f"val_accuracy = {acc:.4f}")

            if acc > best_acc:
                best_acc = acc
                best_params = (d, f)

    print("\nЛучшие параметры:", best_params, "Acc =", best_acc)
    return best_params


best_depth, best_filters = grid_search()


# 5. АНАЛИЗ ВЛИЯНИЯ ГЛУБИНЫ СЕТИ НА КАЧЕСТВО
depths_to_test = [1, 2, 3, 4]
test_acc_results = []

for d in depths_to_test:
    print(f"\nТестирую глубину depth={d}")
    model = build_cnn_model(depth=d, filters=32)
    model.fit(
        X_train, y_train_oh,
        validation_data=(X_val, y_val_oh),
        epochs=3, batch_size=64, verbose=0
    )
    loss, acc = model.evaluate(X_test, y_test_oh, verbose=0)
    test_acc_results.append(acc)
    print(f"Test accuracy: {acc:.4f}")


# График зависимости точности от глубины
plt.figure(figsize=(6,4))
plt.plot(depths_to_test, test_acc_results, marker="o")
plt.title("Влияние глубины CNN на качество (Fashion-MNIST)")
plt.xlabel("Глубина (число Conv-блоков)")
plt.ylabel("Test accuracy")
plt.grid(True)
plt.show()


### Задание 3: Визуализация работы CNN

**Задача:** Исследовать внутренние представления CNN и визуализировать feature maps

**Подзадачи:**
1. Создайте CNN модель с возможностью извлечения промежуточных представлений
2. Визуализируйте feature maps разных слоев
3. Проанализируйте, какие признаки извлекают разные слои сети
4. Исследуйте влияние различных функций активации
5. Сравните feature maps для разных классов данных

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

# 1. ЗАГРУЗКА ДАТАСЕТА
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# нормализация
X_train = X_train.astype("float32")/255.0
X_test = X_test.astype("float32")/255.0

# добавляем канал
X_train = np.expand_dims(X_train, -1)   # (60000, 28, 28, 1)
X_test  = np.expand_dims(X_test, -1)

# 2. СОЗДАНИЕ CNN МОДЕЛИ (с возможностью извлечения feature maps)
def build_visual_cnn(activation="relu"):
    inputs = layers.Input(shape=(28, 28, 1))

    x = layers.Conv2D(32, (3,3), activation=activation, padding="same")(inputs)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(64, (3,3), activation=activation, padding="same")(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(128, (3,3), activation=activation, padding="same")(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation=activation)(x)
    outputs = layers.Dense(10, activation="softmax")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


model = build_visual_cnn()
model.summary()

# быстрое обучение модели
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.1)


# 3. СОЗДАНИЕ МОДЕЛИ ДЛЯ ИЗВЛЕЧЕНИЯ ПРОМЕЖУТОЧНЫХ ПРЕДСТАВЛЕНИЙ

# берём выходы всех сверточных слоёв
layer_outputs = [layer.output for layer in model.layers if "conv" in layer.name]

# модель, которая возвращает список feature maps
activation_model = Model(inputs=model.input, outputs=layer_outputs)


# 4. ВИЗУАЛИЗАЦИЯ FEATURE MAPS
def plot_feature_maps(img, feature_maps, layer_names):
    plt.figure(figsize=(15, 12))
    for i, fmap in enumerate(feature_maps):
        fmap = np.squeeze(fmap)  # remove batch dim
        num_features = fmap.shape[-1]
        size = fmap.shape[0]

        # показываем первые 8 карт признаков
        display_maps = min(8, num_features)

        for j in range(display_maps):
            plt.subplot(len(feature_maps), display_maps, i*display_maps + j + 1)
            plt.imshow(fmap[:, :, j], cmap="viridis")
            plt.axis("off")
            if j == 0:
                plt.ylabel(layer_names[i], fontsize=10)

    plt.suptitle("Feature maps разных сверточных слоев", fontsize=16)
    plt.show()


# пример изображения
img = X_test[0:1]
true_label = y_test[0]

feature_maps = activation_model.predict(img)
layer_names = [layer.name for layer in model.layers if "conv" in layer.name]

plot_feature_maps(img, feature_maps, layer_names)

# 5. АНАЛИЗ: какие признаки извлекают слои?


print("\nАНАЛИЗ:")
print("Слой 1 — выявляет простые границы и текстуры.")
print("Слой 2 — формирует более сложные паттерны (кривые, углы).")
print("Слой 3 — выделяет высокоуровневые структуры (части объектов).")


# 6. ИССЛЕДОВАНИЕ РАЗНЫХ ФУНКЦИЙ АКТИВАЦИИ

activations = ["relu", "tanh", "sigmoid"]
activation_results = {}

for act in activations:
    print(f"\n=== Обучаю CNN с активацией {act} ===")
    m = build_visual_cnn(activation=act)
    history = m.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.1, verbose=0)
    loss, acc = m.evaluate(X_test, y_test, verbose=0)
    activation_results[act] = acc
    print(f"Test accuracy: {acc:.4f}")

print("\nСравнение функций активации:")
print(activation_results)

# 7. СРАВНЕНИЕ FEATURE MAPS ДЛЯ РАЗНЫХ КЛАССОВ
def compare_classes(class1, class2):
    idx1 = np.where(y_test == class1)[0][0]
    idx2 = np.where(y_test == class2)[0][0]

    img1 = X_test[idx1:idx1+1]
    img2 = X_test[idx2:idx2+1]

    fm1 = activation_model.predict(img1)
    fm2 = activation_model.predict(img2)

    print(f"\nСравнение feature maps классов {class1} и {class2}")

    plt.figure(figsize=(14,6))

    # отображаем по 1 карте на слой
    for i, fmap in enumerate(fm1):
        plt.subplot(2, len(fm1), i+1)
        plt.imshow(np.squeeze(fmap)[..., 0], cmap="inferno")
        plt.title(f"{layer_names[i]} class {class1}")
        plt.axis("off")

    for i, fmap in enumerate(fm2):
        plt.subplot(2, len(fm2), len(fm1) + i + 1)
        plt.imshow(np.squeeze(fmap)[..., 0], cmap="inferno")
        plt.title(f"{layer_names[i]} class {class2}")
        plt.axis("off")

    plt.show()


# пример: сравним класс 0 ("T-shirt") и 9 ("boot")
compare_classes(0, 9)



### Задание 4: Сравнение различных подходов к обработке данных

**Задача:** Сравнить эффективность CNN с другими архитектурами нейронных сетей

**Подзадачи:**
1. Реализуйте RNN/LSTM для последовательных данных
2. Создайте автокодировщик для обучения без учителя
3. Сравните CNN с традиционными методами машинного обучения
4. Проанализируйте преимущества и недостатки каждого подхода
5. Определите оптимальные области применения для каждого метода

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import layers, models, callbacks, losses
from tensorflow.keras.utils import to_categorical

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X = np.concatenate([X_train_full, X_test], axis=0)
y = np.concatenate([y_train_full, y_test], axis=0)

# небольшая подвыборка для традиционных методов (иначе SVM будет долго)
SUBSET_FOR_CLASSICAL = 15000  # оставь None для полного набора (но будет медленнее)
if SUBSET_FOR_CLASSICAL:
    idx = np.random.RandomState(42).choice(len(X), size=SUBSET_FOR_CLASSICAL, replace=False)
    X_small = X[idx]
    y_small = y[idx]
else:
    X_small = X
    y_small = y

# нормализация для нейросетей
X_nn = X.astype('float32') / 255.0
X_nn = np.expand_dims(X_nn, -1)  # (N,28,28,1)

# разделение на train/val/test для нейросетей
X_train, X_test_nn, y_train, y_test_nn = train_test_split(X_nn, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, stratify=y_train)
# итог: ~60% train, 10% val, 30% test

print("Shapes NN:", X_train.shape, X_val.shape, X_test_nn.shape)

# для classical ML: плоский вектор + стандартизация / PCA опционально
X_small_flat = X_small.reshape(len(X_small), -1).astype('float32') / 255.0
scaler = StandardScaler()
X_small_scaled = scaler.fit_transform(X_small_flat)

# PCA для уменьшения размерности (ускорение SVM/RF) — сохраняем 100 компонент
pca = PCA(n_components=100, random_state=42)
X_small_pca = pca.fit_transform(X_small_scaled)
print("Classical data shape (PCA):", X_small_pca.shape)


# 1. RNN / LSTM (изображение -> последовательность)
# Подход: каждую строку 28 пикселей считаем timestep, размерность 28 (features). Последовательность длины 28.
X_train_seq = X_train.squeeze(-1)  # (N,28,28)
X_val_seq = X_val.squeeze(-1)
X_test_seq = X_test_nn.squeeze(-1)

# LSTM требует shape (samples, timesteps, features) => (N,28,28)
def build_lstm_model(hidden_units=64):
    inp = layers.Input(shape=(28, 28))
    x = layers.Masking()(inp)  # не обязательно, но оставим
    x = layers.LSTM(hidden_units, return_sequences=False)(x)
    x = layers.Dense(64, activation='relu')(x)
    out = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inp, out)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("\n=> Обучаем LSTM (используем 28 timesteps x 28 features)")
lstm = build_lstm_model(128)
start = time.time()
history_lstm = lstm.fit(X_train_seq, y_train, validation_data=(X_val_seq, y_val), epochs=8, batch_size=128, verbose=1)
lstm_time = time.time() - start
print("LSTM training time: %.1f s" % lstm_time)
loss, acc = lstm.evaluate(X_test_seq, y_test_nn, verbose=0)
print("LSTM test acc:", acc)

# 2. Автокодировщик (unsupervised)
# Преобразуем в плоский вектор для простого dense автокодировщика или используем сверточный автокодировщик.
X_ae_train = X_train  # (N,28,28,1)
X_ae_val = X_val
X_ae_test = X_test_nn

def build_conv_autoencoder(latent_dim=64):
    inp = layers.Input(shape=(28,28,1))
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inp)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Flatten()(x)
    encoded = layers.Dense(latent_dim, activation='relu', name='latent')(x)

    # decoder
    x = layers.Dense(7*7*64, activation='relu')(encoded)
    x = layers.Reshape((7,7,64))(x)
    x = layers.Conv2DTranspose(64, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2DTranspose(32, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.UpSampling2D((2,2))(x)
    decoded = layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

    ae = models.Model(inp, decoded)
    encoder = models.Model(inp, encoded)  # отдельно encoder для получения латентных векторов
    ae.compile(optimizer='adam', loss='mse')
    return ae, encoder

print("\n=> Обучаем сверточный автокодировщик")
ae, encoder = build_conv_autoencoder(latent_dim=64)
ae.fit(X_ae_train, X_ae_train, validation_data=(X_ae_val, X_ae_val), epochs=15, batch_size=256, verbose=1)
# получаем латентные представления для теста
Z_test = encoder.predict(X_ae_test)
print("Latent shape:", Z_test.shape)

# Визуализация примеров реконструкции
n = 6
recon = ae.predict(X_ae_test[:n])
plt.figure(figsize=(10,4))
for i in range(n):
    plt.subplot(2,n,i+1)
    plt.imshow(X_ae_test[i].squeeze(), cmap='gray')
    plt.axis('off')
    plt.subplot(2,n,n+i+1)
    plt.imshow(recon[i].squeeze(), cmap='gray')
    plt.axis('off')
plt.suptitle("Оригинал (верх) vs Реконструкция (низ)")
plt.show()

# 3. Сравнение CNN с традиционными методами
# 3a: CNN (сверточная модель)
def build_cnn_small():
    inp = layers.Input(shape=(28,28,1))
    x = layers.Conv2D(32,(3,3),activation='relu',padding='same')(inp)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(64,(3,3),activation='relu',padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation='relu')(x)
    out = layers.Dense(10,activation='softmax')(x)
    model = models.Model(inp,out)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("\n=> Обучаем базовый CNN")
cnn = build_cnn_small()
start = time.time()
cnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=6, batch_size=128, verbose=1)
cnn_time = time.time() - start
print("CNN training time: %.1f s" % cnn_time)
loss, cnn_acc = cnn.evaluate(X_test_nn, y_test_nn, verbose=0)
print("CNN test acc:", cnn_acc)

# 3b: Традиционные методы (RandomForest, LogisticRegression, SVM) на PCA-признаках
print("\n=> Обучаем классические методы на PCA-данных (subset)")

Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_small_pca, y_small, test_size=0.2, random_state=42, stratify=y_small)

# RandomForest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
start = time.time()
rf.fit(Xc_train, yc_train)
rf_time = time.time() - start
rf_acc = rf.score(Xc_test, yc_test)
print("RF time: %.1f s, acc: %.4f" % (rf_time, rf_acc))

# LogisticRegression (multinomial)
lr = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='saga', n_jobs=-1)
start = time.time()
lr.fit(Xc_train, yc_train)
lr_time = time.time() - start
lr_acc = lr.score(Xc_test, yc_test)
print("LR time: %.1f s, acc: %.4f" % (lr_time, lr_acc))

# SVM (RBF) — может быть медленным
svc = SVC(kernel='rbf', gamma='scale')
start = time.time()
svc.fit(Xc_train, yc_train)
svc_time = time.time() - start
svc_acc = svc.score(Xc_test, yc_test)
print("SVC time: %.1f s, acc: %.4f" % (svc_time, svc_acc))


# 4. Анализ преимуществ и недостатков (подытожим результаты)
print("\n=== Сводная таблица (точности) ===")
results = {
    'Model': ['LSTM(seq)', 'CNN', 'Autoencoder+Linear', 'RandomForest', 'LogisticRegression', 'SVM-RBF'],
    'Test accuracy': [
        round(float(acc),4),                      # LSTM acc variable 'acc'
        round(float(cnn_acc),4),
        None,  # заполнится ниже: we'll train a linear classifier on AE latents
        round(float(rf_acc),4),
        round(float(lr_acc),4),
        round(float(svc_acc),4)
    ],
    'Train time (s)': [round(lstm_time,1), round(cnn_time,1), None, round(rf_time,1), round(lr_time,1), round(svc_time,1)]
}

# 4a: Автокодировщик + линейный классификатор на латентных признаках
# берем латентные вектора из encoder для подвыборки X_small
print("\n=> Обучаем LogisticRegression на латентных векторах AE (вся тестовая выборка)")
# получим латенты для небольшого поднабора (чтобы не было долго)
sub_idx = np.random.RandomState(1).choice(len(X), size=10000, replace=False)
X_sub = X[sub_idx].astype('float32') / 255.0
X_sub = np.expand_dims(X_sub, -1)
y_sub = y[sub_idx]

Z_sub = encoder.predict(X_sub, batch_size=256)
Z_train, Z_test_lat, yZ_train, yZ_test = train_test_split(Z_sub, y_sub, test_size=0.2, random_state=42, stratify=y_sub)

lr_ae = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='saga', n_jobs=-1)
start = time.time()
lr_ae.fit(Z_train, yZ_train)
lr_ae_time = time.time() - start
lr_ae_acc = lr_ae.score(Z_test_lat, yZ_test)
print("LogReg on AE-latent: time %.1f s acc %.4f" % (lr_ae_time, lr_ae_acc))

# добавляем в results
results['Test accuracy'][2] = round(float(lr_ae_acc),4)
results['Train time (s)'][2] = round(lr_ae_time,1)

df_results = pd.DataFrame(results)
print(df_results)

# 5. Доп. оценки: confusion matrix и classification report для CNN и RF
print("\n=== Classification report: CNN (test) ===")
y_pred_cnn = np.argmax(cnn.predict(X_test_nn), axis=1)
print(classification_report(y_test_nn, y_pred_cnn))

print("\n=== Confusion matrix: CNN ===")
print(confusion_matrix(y_test_nn, y_pred_cnn))

print("\n=== Classification report: RandomForest (on PCA subset test) ===")
yc_pred_rf = rf.predict(Xc_test)
print(classification_report(yc_test, yc_pred_rf))


# Сохраняем результаты в CSV
df_results.to_csv("comparison_results.csv", index=False)
print("\nSaved results to comparison_results.csv")


2025-12-06 19:18:30.010209: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/mac/Documents/Intilect/Intelligence/Practical work №3/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Shapes NN: (49000, 28, 28, 1) (7000, 28, 28, 1) (14000, 28, 28, 1)
Classical data shape (PCA): (15000, 100)

=> Обучаем LSTM (используем 28 timesteps x 28 features)
Epoch 1/8
383/383 ━━━━━━━━━━━━━━━━━━━━ 41s 99ms/step - accuracy: 0.6516 - loss: 0.9914 - val_accuracy: 0.8070 - val_loss: 0.5118
Epoch 2/8
383/383 ━━━━━━━━━━━━━━━━━━━━ 40s 104ms/step - accuracy: 0.8271 - loss: 0.4717 - val_accuracy: 0.8453 - val_loss: 0.4231
Epoch 3/8
383/383 ━━━━━━━━━━━━━━━━━━━━ 41s 108ms/step - accuracy: 0.8455 - loss: 0.4091 - val_accuracy: 0.8484 - val_loss: 0.4036
Epoch 4/8
383/383 ━━━━━━━━━━━━━━━━━━━━ 44s 115ms/step - accuracy: 0.8640 - loss: 0.3674 - val_accuracy: 0.8636 - val_loss: 0.3658
Epoch 5/8
383/383 ━━━━━━━━━━━━━━━━━━━━ 42s 110ms/step - accuracy: 0.8699 - loss: 0.3487 - val_accuracy: 0.8670 - val_loss: 0.3702
Epoch 6/8
383/383 ━━━━━━━━━━━━━━━━━━━━ 50s 131ms/step - accuracy: 0.8745 - loss: 0.3388 - val_accuracy: 0.8686 - val_loss: 0.3497
Epoch 7/8
383/383 ━━━━━━━━━━━━━━━━━━━━ 43s 112ms/step - 

### Теоретические вопросы:

1. В чем преимущества CNN перед полносвязными сетями для обработки структурированных данных?
2. Какие типы данных наиболее подходят для анализа с помощью CNN?
3. Как функция пулинга влияет на работу сверточной сети?
4. В чем разница между 1D, 2D и 3D свертками?
5. Какие методы регуляризации наиболее эффективны для CNN?


ВАШ ОТВЕТ НА ВОПРОСЫ:


1. CNN используют локальные рецептивные поля, поэтому улавливают пространственные закономерности.
Количество параметров значительно меньше, чем у полносвязных слоев.
Есть параметрическое разделение весов (один фильтр используется по всему изображению).
Лучше обобщают на данных с локальными паттернами (изображения, временные ряды).
2. CNN используют локальные рецептивные поля, поэтому улавливают пространственные закономерности.
Количество параметров значительно меньше, чем у полносвязных слоев.
Есть параметрическое разделение весов (один фильтр используется по всему изображению).
Лучше обобщают на данных с локальными паттернами (изображения, временные ряды).
3. Изображения (2D, 3D).
Временные ряды или последовательности (1D-CNN).
Видео (3D-CNN).
Любые данные с локальной структурой, например спектрограммы аудио.
4. Уменьшает размерность активаций → снижает вычисления.
Делает сеть более инвариантной к небольшим смещениям.
Помогает выделять наиболее значимые признаки (например, max-pool берёт самое сильное локальное активирование).
5. 1D свертка: применяется по одному измерению (время, последовательности).
2D свертка: применяется по двум пространственным измерениям (изображения).
3D свертка: применяется по трём измерениям (видео, объемные данные, MRI).